# Decodificando la señal de un HC-12
Con SigDigger capturamos varias tramas, las cuales se guardan como un stream de 1 y 0

Abrimos el archivo y lo almacenamos como un string, luego lo convertimos a una lista, el objetivo es detectar la ocurrencia del preambulo y conocer la posición de cada trama 

In [ ]:
file = open('hc12_00.txt', mode='r')
entrada = file.read()
file.close()

Convertimos el string en un vector

In [ ]:
entradaVector=list(map(int,str(entrada)))

Buscamos el preámbulo y parte de la palabra de sincronismo, la lista de salida indica la posición donde comienzan las tramas, descartamos los ultimos bits ya que la trama puede estar incompleta.

In [ ]:
pattern = [1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0]
cuantos=[]
N=len(pattern)
M=len(entradaVector)
for i in range(1,M-N,1):
    if entradaVector[i:(i+int(len(pattern)))]==pattern:
        cuantos.append(i)
print(len(cuantos))
cuantos.pop()

Ubicamos en una matriz de numpy las tramas, de esta forma tenemos una matriz donde todas las tramas están ordenadas, opcionalmente podemos hacer un heatmap binario como en SigDigger. Para continuar con el análisis debemos convertir a hexadecimal para que sea mas facil encontrar patrones visualmente.

In [ ]:
import numpy as np
# el ancho lo podemos fijar o utilizar una regla, por ej len(cuantos[2])-len(cuantos[3]) es la longitud de cada trama
# hasta que ocurre nuevamente el preambulo.
ancho = 248
salida = np.empty((0,ancho),int)
for i in cuantos:
    n=int(i)+ancho
    out = list(entradaVector[i:n])
    salida = np.append(salida,[out],axis=0)

Gráfica de un heatmap binario

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

fig,ax = plt.subplots()
cmap = mpl.colors.ListedColormap(['w','k'])
bounds = [0.,0.5,1.]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
ax.imshow(salida, interpolation='none', cmap=cmap, norm=norm)
# si queremos exportar la gráfica
#plt.savefig('trama_01.png')

Guardamos la matriz como un archivo de texto, hasta aqui podríamos tener un script que dado un stream binario devuelva lineas con tramas a decodificar

In [ ]:
np.savetxt(r'hc12_00.bin',salida,fmt='%d',delimiter='')

# TODO
analisis gráfico usando Binwalk, [Vix](http://actinid.org/vix/) , [Binglide](https://github.com/wapiflapi/binglide), [Veles](https://github.com/codilime/veles), [Cantordust](https://github.com/Battelle/cantordust)

# Convirtiendo a hexadecimal las tramas
Abrimos el archivo de salida anterior

In [ ]:
with open('hc12_00.bin', 'r') as file:
    data2 = file.read().splitlines()
file.close()

Convirtiendo un string binario a un string hexadecimal

In [ ]:
hex(int(data2[0],2))[2:]

Usemos esto para convertir todo a hex

In [ ]:
data2hex = []
for i in range(len(data2)):
    data2hex.append(hex(int(data2[i],2))[2:])
# podría verse mas bonito con un f-string
# print(data2hex)

for i in range(1,10):
    print(f'{data2hex[1]}')


In [ ]:
a="preambulo"
b="sync?"
c="data"
d="crc?"
print(f' {a:^12} | {b:^8} | {c:^36} | {d:^6}')
for i in range(1,10):
    print(f"{data2hex[i][1:12]} | {data2hex[i][12:20]} | {data2hex[i][20:56]} | {data2hex[i][56:-3]}")

# Capturando una sola trama

In [ ]:
raw = 0b101010101010101010101010101010101010101010101010100100011001000110011111101100000011110011011100110011101011110001000101001001010010101000110010111111010111110010011101100100110010100101001111110101011101101101101011110111011001
rawbin=hex(int(raw))[2:]

print(f"{rawbin[1:12]} | {rawbin[12:20]} | {rawbin[20:56]} | {rawbin[56:]}")

La suma exclusiva entre el payload y la llave devuelve el mensaje original

In [ ]:
hex(0x3cdccebc45252a32fd7c9d93294fd5db6bdd ^ 0x59b7a1cc24575e4b9c0ee9ea502abeb41bb6)

In [ ]:
chr(0x3c^0x59)+chr(0xdc^0xb7)+chr(0xce^0xa1)

#                           DEMO

In [180]:
import serial
ser = serial.Serial('/dev/ttyUSB0', 9600, timeout=0.5)
ser.write(" eko16 pwndemic ".encode())
ser.close()

In [181]:
def change_to_be_hex(str):
    return int(str,base=16)

def xor_two_str(str1,str2):
    a = change_to_be_hex(str1)
    b = change_to_be_hex(str2)
    return hex(a ^ b)

msg_raw=[]
with open('demo.txt', 'r') as file:
    msg_raw = file.read()
file.close()
key="59b7a1cc24575e4b9c0ee9ea502abeb41bb6"
msg = hex(int(msg_raw[:-(len(msg_raw)%8)],2))[22:(22+len(key))]
xor_msg_key=xor_two_str(msg,key)[2:]
bytes_object = bytes.fromhex(xor_msg_key)
print(bytes_object)
#ascii_string = bytes_object.decode("ASCII")
#print(ascii_string)


b' eko16 pwndemic \xbaK'


# TODO
* probar denegación de servicio con payloads que puedan confundir al packet handler
* script para generar los payloads

Los payload pueden ser tal que envien constantemente 0xaa 0x00 y 0xff

In [ ]:
payload = 16*"00"
key = 0x59b7a1cc24575e4b9c0ee9ea502abeb41bb6
msg=int("0x"+payload,16)
hex( msg ^ key)

In [ ]:
payload = 16*"aa"
key = 0x59b7a1cc24575e4b9c0ee9ea502abeb41bb6
msg=int("0x"+payload,16)
hex( msg ^ key)

In [ ]:
import serial
ser = serial.Serial('/dev/ttyUSB1', 9600, timeout=0.5)
#59 b7 a1 cc 24 57 5e 4b 9c 0e e9 ea 50 2a be b4 1b b6
command = [b'\x59',b'\xb7',b'\xa1',b'\xcc',b'\x24',b'\x57',b'\x5e',b'\x4b',b'\x9c',b'\x0e',b'\xe9',b'\xea',b'\x50',b'\x2a',b'\xbe',b'\xb4',b'\x1b',b'\xb6']
for i in range(len(command)):
    ser.write(command[i])
ser.close()